# Benchmarking foldseek ctranslate2 branch
Apply the [foldseek papaer](https://doi.org/10.1038/s41587-023-01773-0) benchmark using the `ctranslate2` branch.
The paper uses the [SCOPe](https://wwwuser.gwdg.de/~compbiol/foldseek/scop40pdb.tar.gz) 40% sequence redundance database.
However, we don't download the data here, but reuse existing `.pdb` data on the luster at `/home/sukhwan/foldseek-analysis/scope_pdb`.

In [ ]:
mkdir -p test/data/scope
ln -s /home/sukhwan/foldseek-analysis/scope_pdb test/data/scope/pdb

In [ ]:
# create foldseek db
srun -c 32 -t 20-0 --pty /bin/bash
DB_PATH=test/dbs/scope/
SCOPE_DATA=test/data/scope/pdb
mkdir -p $DB_PATH
# conda activate prostt5
# foldseek createdb --mask-bfactor-threshold -10000 $SCOPE_DATA $DB_PATH/scope_full
foldseek createdb $SCOPE_DATA $DB_PATH/scope_full

foldseek convert2fasta $DB_PATH/scope_full $(dirname $SCOPE_DATA)/scope_full.aa.fasta

In [ ]:
# create foldseek db from amino acid fasta using ProstT5
srun -p gpu --gres=gpu:3 -c 9 -t 20-0 --pty /bin/bash
MODEL_PATH=/home/sukhwan/foldseek_ctranslate/foldseek/weights/model
DB_PATH=test/dbs/scope

for SPLIT_LENGTH in 256 512 1024; do
# select subset of scope that is affected by splitting
awk -v s=$SPLIT_LENGTH '$3 > s {print $1;}' $DB_PATH/scope_full.index > $DB_PATH/scope_gt${SPLIT_LENGTH}_dbkeys.tsv;

# subset dbs
foldseek createsubdb $DB_PATH/scope_gt${SPLIT_LENGTH}_dbkeys.tsv $DB_PATH/scope_full $DB_PATH/scope_gt${SPLIT_LENGTH};
foldseek createsubdb $DB_PATH/scope_gt${SPLIT_LENGTH}_dbkeys.tsv $DB_PATH/scope_full_ss $DB_PATH/scope_gt${SPLIT_LENGTH}_ss;
foldseek createsubdb $DB_PATH/scope_gt${SPLIT_LENGTH}_dbkeys.tsv $DB_PATH/scope_full_ca $DB_PATH/scope_gt${SPLIT_LENGTH}_ca;

# create aa.fasta
foldseek convert2fasta $DB_PATH/scope_gt${SPLIT_LENGTH} $DB_PATH/scope_gt${SPLIT_LENGTH}.aa.fasta

# cpu-build/src/foldseek createdb test/data/scope/scope_full.aa.fasta test/dbs/scope/scope_prostt5 --prostt5-model $MODEL_PATH
done
CUDA_VISIBLE_DEVICES=0 cuda-build/src/foldseek createdb $DB_PATH/scope_gt256.aa.fasta $DB_PATH/scope_pt5_gt256 --prostt5-model $MODEL_PATH --gpu 1 --prostt5-split-length 256 --threads 3 &
CUDA_VISIBLE_DEVICES=1 cuda-build/src/foldseek createdb $DB_PATH/scope_gt512.aa.fasta $DB_PATH/scope_pt5_gt512 --prostt5-model $MODEL_PATH --gpu 1 --prostt5-split-length 512 --threads 3 &
CUDA_VISIBLE_DEVICES=2 cuda-build/src/foldseek createdb $DB_PATH/scope_gt1024.aa.fasta $DB_PATH/scope_pt5_gt1024 --prostt5-model $MODEL_PATH --gpu 1 --prostt5-split-length 1024 --threads 3 &
wait

exit

In [ ]:
# create subsets of lookup file
for SPLIT_LENGTH in 256 512 1024

#TODO

In [ ]:
# conda activate prostt5
srun -c 64 -t 20-0 --pty /bin/bash
DB_PATH=test/dbs/scope/
mkdir -p $DB_PATH/tmp

# alternatively clone git@github.com:steineggerlab/foldseek-analysis.git
AWK_SCRIPT=../../../consensus3Di/lib/foldseek-analysis/scopbenchmark/scripts/bench.noselfhit.awk
LOOKUP_FILE=../../../consensus3Di/lib/foldseek-analysis/scopbenchmark/data/scop_lookup.fix.tsv
NO_SPLIT_ROCX_FILE=test/data/benchmark/ctranslate2.scope.no_splitting.rocx

for SPLIT_LENGTH in 256 512 1024; do
foldseek search $DB_PATH/scope_pt5_gt${SPLIT_LENGTH} $DB_PATH/scope_full $DB_PATH/aln_scope_gt${SPLIT_LENGTH}_benchmark $DB_PATH/tmp/ -s 9.5 --max-seqs 2000 -e 10
foldseek convertalis $DB_PATH/scope_pt5_gt${SPLIT_LENGTH} $DB_PATH/scope_full $DB_PATH/aln_scope_gt${SPLIT_LENGTH}_benchmark $DB_PATH/aln_scope_gt${SPLIT_LENGTH}_benchmark.m8

for SPLIT_LENGTH_2 in 256 512 1024; do
# roc performance metrics
SPLIT_ROCX_FILE=test/data/benchmark/ctranslate2.scope.gt${SPLIT_LENGTH}.rocx
$AWK_SCRIPT $LOOKUP_FILE <(cat $DB_PATH/aln_scope_gt${SPLIT_LENGTH}_benchmark.m8 | sed -E 's|.pdb||g') > $SPLIT_ROCX_FILE
done

# filter sequences affected from splitting and combine the data from unaffected sequences form $NO_SPLIT_ROCX_FILE and affected sequences from $SPLIT_ROCX_FILE
SPLIT_MERGED_ROCX_FILE=$(dirname $SPLIT_ROCX_FILE)/$(basename $SPLIT_ROCX_FILE .rocx).merged.rocx
awk 'NR == FNR { f[$1]=$0; next } !($1 in f) { print; }' $SPLIT_ROCX_FILE $NO_SPLIT_ROCX_FILE | cat - $SPLIT_ROCX_FILE > $SPLIT_MERGED_ROCX_FILE

# compute roc values for family, superfamily and fold
awk '{ famsum+=$3; supfamsum+=$4; foldsum+=$5}END{print famsum/NR,supfamsum/NR,foldsum/NR}' $SPLIT_MERGED_ROCX_FILE > $SPLIT_MERGED_ROCX_FILE.values
done

# No splitting ROC values (Fam, Superfam, Fold)
# 0.841243 0.444564 0.0849547

exit

In [ ]:
# TODO: merge .rocx files with test/data/benchmark.ctranslate2.splitting.rocx, which holds the values for sequences unaffected by splitting

for SPLIT_LENGTH in 256 512 1024; do
SPLIT_ROCX_FILE=test/data/benchmark/ctranslate2.scope.gt${SPLIT_LENGTH}.rocx
SPLIT_MERGED_ROCX_FILE=$(dirname $SPLIT_ROCX_FILE)/$(basename $SPLIT_ROCX_FILE .rocx).merged.rocx

# filter sequences affected from splitting and combine the data from unaffected sequences form $NO_SPLIT_ROCX_FILE and affected sequences from $SPLIT_ROCX_FILE
awk 'NR == FNR { f[$1]=$0; next } !($1 in f) { print; }' $SPLIT_ROCX_FILE $NO_SPLIT_ROCX_FILE | cat - $SPLIT_ROCX_FILE > $SPLIT_MERGED_ROCX_FILE

# compute roc values for family, superfamily and fold
awk '{ famsum+=$3; supfamsum+=$4; foldsum+=$5}END{print famsum/NR,supfamsum/NR,foldsum/NR}' $SPLIT_MERGED_ROCX_FILE
done

# 0.304635 0.162114 0.028663
# 0.307835 0.167328 0.0309218
# 0.307732 0.167367 0.0309591
